In [6]:
import pandas as pd
import numpy as np

In [7]:
m = pd.read_csv("../../data/processed/KOBIS_개봉일람_2.csv")

m

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화
4,어벤져스: 엔드게임,안소니 루소조 루소,마블 스튜디오,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,늑대와 빨간 재킷,뱅상 파로노,NaN,판씨네마(주),판씨네마(주),2021-02-03,개봉영화,장편,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화
3971,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,(주)영화사오원,(주)영화사오원,2021-01-07,개봉영화,장편,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화
3972,락다운 213주,아담 메이슨,플래티넘 듄스,그린나래미디어(주),그린나래미디어(주)(주)인터파크,2022-08-31,개봉영화,장편,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화
3973,래미의 드래곤월드 구출작전,웨이밍 황,NaN,와이드 릴리즈(주),와이드 릴리즈(주),2020-07-16,개봉영화,장편,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화


In [8]:
m.isna().sum()

영화명           0
감독            6
제작사        1643
수입사        1272
배급사           0
개봉일           0
영화유형          0
영화형태          0
국적            0
전국 스크린수       0
전국 매출액        0
전국 관객수        0
서울 매출액        0
서울 관객수        0
장르            0
등급            0
영화구분          0
dtype: int64

In [9]:
m1 = m[["전국 관객수"]].groupby(m["감독"]).sum()
m1.loc[m1["전국 관객수"] == 0, "전국 관객수"] = 1
m1 = m1.apply(np.log).reset_index()
m1.rename(columns={"전국 관객수":"감독_로그"}, inplace=True)
m1

,감독,감독_로그
0,D.J. 카루소,15.037578
1,F. 게리 그레이,15.494588
2,J 블레이크슨,12.002799
3,J.C 챈더,9.061144
4,J.J. 에이브럼스,15.757183
...,...,...
2522,히라카와 유이치로,8.935640
2523,히로키 류이치,12.164625
2524,히로타 유스케,10.073947
2525,히시다 마사카즈,11.926630


In [10]:
m = pd.merge(m, m1, how="left", left_on="감독", right_on="감독")
m

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,감독_로그
0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화,17.311865
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화,16.835596
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화,17.412296
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화,17.181630
4,어벤져스: 엔드게임,안소니 루소조 루소,마블 스튜디오,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화,17.337555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,늑대와 빨간 재킷,뱅상 파로노,NaN,판씨네마(주),판씨네마(주),2021-02-03,개봉영화,장편,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화,6.856462
3971,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,(주)영화사오원,(주)영화사오원,2021-01-07,개봉영화,장편,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화,11.603716
3972,락다운 213주,아담 메이슨,플래티넘 듄스,그린나래미디어(주),그린나래미디어(주)(주)인터파크,2022-08-31,개봉영화,장편,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화,6.739337
3973,래미의 드래곤월드 구출작전,웨이밍 황,NaN,와이드 릴리즈(주),와이드 릴리즈(주),2020-07-16,개봉영화,장편,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화,8.386857


In [11]:
m1 = m[["전국 관객수"]].groupby(m["제작사"]).sum()
m1.loc[m1["전국 관객수"] == 0, "전국 관객수"] = 1
m1 = m1.apply(np.log).reset_index()
m1.rename(columns={"전국 관객수":"제작사_로그"}, inplace=True)
m1

,제작사,제작사_로그
0,(Europe) Media Program,11.115458
1,(유)내부자들 문화전문회사,16.029894
2,(유)박열문화전문회사,14.673822
3,(유)이층의 악당 문화산업전문회사,13.313354
4,(재)CBS,11.126351
...,...,...
1438,휠므빠말주식회사이십세기폭스 인터내셔널 프로덕션,11.802254
1439,휴메니테라픽쳐스(주)컴퍼니에이이엔티(주),10.382141
1440,히어로 엔터테인먼트,10.850404
1441,히트 엔터테인먼트니트로겐 스튜디오,9.691902


In [12]:
m = pd.merge(m, m1, how="left", left_on="제작사", right_on="제작사")
m

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,감독_로그,제작사_로그
0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화,17.311865,17.069823
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화,16.835596,16.604523
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화,17.412296,17.098825
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화,17.181630,17.522923
4,어벤져스: 엔드게임,안소니 루소조 루소,마블 스튜디오,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화,17.337555,18.198196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,늑대와 빨간 재킷,뱅상 파로노,NaN,판씨네마(주),판씨네마(주),2021-02-03,개봉영화,장편,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화,6.856462,NaN
3971,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,(주)영화사오원,(주)영화사오원,2021-01-07,개봉영화,장편,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화,11.603716,NaN
3972,락다운 213주,아담 메이슨,플래티넘 듄스,그린나래미디어(주),그린나래미디어(주)(주)인터파크,2022-08-31,개봉영화,장편,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화,6.739337,14.144999
3973,래미의 드래곤월드 구출작전,웨이밍 황,NaN,와이드 릴리즈(주),와이드 릴리즈(주),2020-07-16,개봉영화,장편,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화,8.386857,NaN


In [13]:
m1 = m[["전국 스크린수"]].groupby(m["배급사"]).sum()
m1.loc[m1["전국 스크린수"] == 0, "전국 스크린수"] = 1
m1 = m1.apply(np.log).reset_index()
m1.rename(columns={"전국 스크린수":"배급사_로그"}, inplace=True)
m1

,배급사,배급사_로그
0,(유)씨에이치엔터테인먼트,5.752573
1,(유)영화사 화수분(주)에스비에스콘텐츠허브,5.488938
2,(유)조이앤시네마,6.381816
3,(유)조이앤시네마(주)제이앤씨미디어그룹,4.369448
4,(재)CBS,6.359574
...,...,...
514,필라멘트픽쳐스,8.049746
515,필름다빈,5.463832
516,한국소니픽쳐스릴리징브에나비스타영화㈜,9.993054
517,홀리가든,4.189655


In [14]:
m = pd.merge(m, m1, how="left", left_on="배급사", right_on="배급사")
m

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,감독_로그,제작사_로그,배급사_로그
0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,1.357484e+11,17613682,3.312123e+10,4163666,사극,15세이상관람가,일반영화,17.311865,17.069823,11.922104
1,극한직업,이병헌,(주)어바웃잇영화사 해그림 주식회사(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,1.396480e+11,16264944,3.185866e+10,3638287,코미디,15세이상관람가,일반영화,16.835596,16.604523,11.922104
2,신과함께-죄와 벌,김용화,리얼라이즈픽쳐스(주)(주)덱스터스튜디오,NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,1.156987e+11,14410754,2.753083e+10,3346172,판타지,12세이상관람가,일반영화,17.412296,17.098825,11.433168
3,국제시장,윤제균,(주)제이케이필름(주)씨제이이엔엠,NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,1.108280e+11,14245998,2.584252e+10,3233946,드라마,12세이상관람가,일반영화,17.181630,17.522923,11.922104
4,어벤져스: 엔드게임,안소니 루소조 루소,마블 스튜디오,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,개봉영화,장편,미국,2835,1.221827e+11,13934592,3.357714e+10,3597963,액션,12세이상관람가,일반영화,17.337555,18.198196,11.297552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,늑대와 빨간 재킷,뱅상 파로노,NaN,판씨네마(주),판씨네마(주),2021-02-03,개봉영화,장편,프랑스,74,8.525200e+06,950,2.893740e+06,345,스릴러,청소년관람불가,독립/예술영화,6.856462,NaN,9.680281
3971,마라가 큰 결정을 해야 해,로버트 루케틱,NaN,(주)영화사오원,(주)영화사오원,2021-01-07,개봉영화,장편,미국,79,7.475940e+06,856,2.830800e+06,321,멜로/로맨스,15세이상관람가,일반영화,11.603716,NaN,7.836370
3972,락다운 213주,아담 메이슨,플래티넘 듄스,그린나래미디어(주),그린나래미디어(주)(주)인터파크,2022-08-31,개봉영화,장편,미국,61,7.664799e+06,845,3.573900e+06,381,스릴러,15세이상관람가,일반영화,6.739337,14.144999,6.023448
3973,래미의 드래곤월드 구출작전,웨이밍 황,NaN,와이드 릴리즈(주),와이드 릴리즈(주),2020-07-16,개봉영화,장편,중국,65,6.067340e+06,779,6.345000e+05,74,애니메이션,전체관람가,일반영화,8.386857,NaN,9.637306


In [15]:
m.to_csv("../../data/processed/KOBIS_개봉일람_3.csv", index=False)